In [1]:
import random
import copy
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean, variance, stdev
from scipy.stats import binom
from collections import Counter
from itertools import chain
from matplotlib import animation
from matplotlib import rcParams
rcParams['animation.embed_limit'] = 1000 #MB
from IPython.display import HTML
from multiprocessing import Pool



In [2]:
# ------------------------------------------------------------------------------------------
POPSIZE = 200

GENOME_LENGTH = 5
MUTATION_RATE = 1/(2*GENOME_LENGTH) #per site rate
GENERATIONS = 50000
TSIZE = -1.3 #negative sets roulette exponent
# ------------------------------------------------------------------------------------------

In [3]:
DR = {}
ER = {}
def getDriftReference(initialPop,finalPop):
    key = (initialPop,finalPop)
    if key not in DR:
        DR[key] = [binom.pmf(x, finalPop, 1/initialPop) for x in range(finalPop+1)]
    return DR[key]

def getEliteReference(initialPop,finalPop):
    key = (initialPop,finalPop)
    if key not in ER:
        ER[key] = [initialPop-1]+[0 for _ in range(finalPop-1)]+[1]
    return ER[key]

In [4]:
class Organism:
    def __init__(self,startingFitness = None,timeBorn = 0):
        global GENOME_LENGTH
        self.genome = [random.randint(0,1) for _ in range(GENOME_LENGTH)]
        self.offspringCount = 0
        self.fitness = startingFitness
        self.timeBorn = timeBorn
        self.timeDied = None
        
    def make_mutated_copy(self,gen=-1):
        global GENOME_LENGTH, MUTATION_RATE
        self.offspringCount += 1
        child = Organism(timeBorn = gen)
        child.genome = copy.deepcopy(self.genome)
        for i in range(GENOME_LENGTH):
            if random.random() <= MUTATION_RATE:
                PN = random.randint(0,1)
                child.genome[i] += (-1*PN)+(1-PN)
        return child
    
    def kill(self,gen):
        self.timeDied = gen
    
    def __repr__(self,):
        return str(self.fitness)


def fitness(org):
    if not org.fitness:
#         org.fitness = 1
        org.fitness = eval_sawTooth(sum(org.genome))
#         org.fitness = max( sum(org.genome), 0)
    return org.fitness


def eval_sawTooth(x):
#     x = org.genomeValue #0 is top of first peak
    w = 6 #valley width
    d = 10 #valley depth
    r = 10 #fitness rise peak to peak
    x = x + w+1 #offset to next peak to avoid fitness zero on init
    return x*(-d/w) + (x//(w+1))*(r + d + (d/w)) 



def roulette_wheel(population,exponent=1.05):
    MAX = max([fitness(x) for x in population])
    F = [np.power(exponent,fitness(x)-MAX) for x in population]
    S = sum(F)
    return [f/S for f in F]




def emd(P,Q):
    assert len(P) == len(Q)
    EMD = [0]
    for i in range(len(P)):
        EMD.append(P[i]-Q[i]+EMD[-1])
    return sum([abs(d) for d in EMD])


def getSS(offCounts_1d,initialPop,finalPop):
    Counts = Counter(offCounts_1d)
#     largest = sorted(Counts.items())[-1][0]
    Observed = [Counts[x]/initialPop if x in Counts else 0 for x in range(finalPop+1)]
    return (emd(getDriftReference(initialPop,finalPop),Observed),
            emd(getEliteReference(initialPop,finalPop),Observed))


In [5]:
def newPopulation(population,gen,popSize,tournament = 0):
    
    if tournament < 0:
        #do roulette
        wheel = roulette_wheel(population)
        child = random.choices(population,k=1,weights=wheel)[0]
        new_pop = population[:] + [child]
        new_pop[random.randint(0,POPSIZE-1)].kill(gen)
    return new_pop
    

def run():
    
    population = [Organism() for _ in range(POPSIZE)]
    popLog = [population]
    for generation in range(GENERATIONS):
        print(100*generation/GENERATIONS,end="\r")

        population = newPopulation(population,generation,POPSIZE,tournament=TSIZE)

        popLog.append(population)

    return popLog


In [6]:
random.seed(6)

popLog = run()



In [7]:
fitnessLog = [org.fitness for org in population if org.timeDied == None]

with Pool(16) as MPPOOL:
    SSdata = MPPOOL.starmap(getSS,zip(offCounts,initialPopSizes,finalPopSizes))
    AVEdata = MPPOOL.map(mean,fitnessLog)
    VARdata = MPPOOL.map(variance,fitnessLog)

driftDists = list(zip(*SSdata))[0]
eliteDists = list(zip(*SSdata))[1]

plt.plot(AVEdata)
plt.show()

NameError: name 'PSlog_T' is not defined